In [101]:
import os
import json
import numpy as np
from collections import defaultdict, Counter

In [6]:
# load model results
mod21_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_50_enc_max_pool_aux_loss/'
mod22_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_43_obj_instance_enc_max_pool_aux_loss/'
mod22_wt11_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce/'
mod23_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_biattn_axu_loss/'

In [3]:
# select by best BLEU score

seen_21_epoch = 11
seen_22_epoch = 11
seen_22_wt11_epoch = 29
seen_23_epoch = 25

unseen_21_epoch = 6
unseen_22_epoch = 12
unseen_22_wt11_epoch = 29
unseen_23_epoch = 25

In [7]:
mod22_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_43_obj_instance_enc_max_pool_aux_loss/'

with open(mod22_dout + 'train_sanity_argmax.debug_epoch_11.preds.json', 'r') as f:
    mod22_train_sanity = json.load(f)
    
with open(mod22_dout + 'valid_seen_argmax.debug_epoch_11.preds.json', 'r') as f:
    mod22_valid_seen = json.load(f)
    
with open(mod22_dout + 'valid_unseen_argmax.debug_epoch_12.preds.json', 'r') as f:
    mod22_valid_unseen = json.load(f)

In [104]:
res = mod22_train_sanity

all_tp_stc_obj_tokens = []
all_tp_vis_obj_tokens = []

for task_id in list(res.keys()):
    
    num_subgoals = len(res[task_id]['action_high'])-1
    
    obj_token_id = np.empty((num_subgoals, len(res[task_id]['obj_token_id'][str(0)])))
    p_obj_vis = np.empty((num_subgoals, len(res[task_id]['p_obj_vis'][str(0)])))
    p_state_change = np.empty((num_subgoals, len(res[task_id]['p_state_change'][str(0)])))
    l_obj_vis = np.empty((num_subgoals, len(res[task_id]['l_obj_vis'][str(0)])))
    l_state_change = np.empty((num_subgoals, len(res[task_id]['l_state_change'][str(0)])))
    assert p_obj_vis.shape == p_state_change.shape == l_obj_vis.shape == l_state_change.shape

    for i in range(num_subgoals):
        obj_token_id[i] = res[task_id]['obj_token_id'][str(i)] 
        p_obj_vis[i] = res[task_id]['p_obj_vis'][str(i)] 
        p_state_change[i] = res[task_id]['p_state_change'][str(i)]
        l_obj_vis[i] = res[task_id]['l_obj_vis'][str(i)]
        l_state_change[i] = res[task_id]['l_state_change'][str(i)]
        
    p_obj_vis = (p_obj_vis > 0.5).astype(np.float32)
    p_state_change = (p_state_change > 0.5).astype(np.float32)
    
#     correct_vis = (l_obj_vis == p_obj_vis).astype(np.float32)
#     correct_stc = (l_state_change == p_state_change).astype(np.float32)
    
    tp_stc = ((l_state_change == 1.0) & (p_state_change == 1.0)).astype(np.float32)
    tp_vis = ((l_obj_vis == 1.0) & (p_obj_vis == 1.0)).astype(np.float32)
    
    tp_stc_pos = np.nonzero(tp_stc)
    tp_vis_pos = np.nonzero(tp_vis)
    
    obj_token_tp_stc = obj_token_id[tp_stc_pos]
    obj_token_tp_vis = obj_token_id[tp_vis_pos]
    all_tp_stc_obj_tokens.extend(obj_token_tp_stc.tolist())
    all_tp_vis_obj_tokens.extend(obj_token_tp_vis.tolist())

In [106]:
Counter(all_tp_stc_obj_tokens).most_common()

[(37.0, 19),
 (1.0, 18),
 (4.0, 8),
 (52.0, 4),
 (33.0, 3),
 (32.0, 3),
 (45.0, 3),
 (63.0, 3),
 (48.0, 2),
 (14.0, 1)]

In [107]:
import torch
object_vocab = torch.load('/root/data_alfred/json_feat_2.1.0/objects_20200522.vocab')

In [109]:
object_vocab['object_type'].index2word(37)

'tomato'

In [110]:
object_vocab['object_type'].index2word(1)

'apple'

In [111]:
object_vocab['object_type'].index2word(4)

'butterknife'

In [112]:
object_vocab['object_type'].index2word(52)

'remotecontrol'

In [113]:
object_vocab['object_type'].index2word(33)

'spoon'

In [114]:
object_vocab['object_type'].index2word(32)

'spatula'

In [115]:
object_vocab['object_type'].index2word(45)

'creditcard'

In [116]:
object_vocab['object_type'].index2word(63)

'cellphone'

In [117]:
object_vocab['object_type'].index2word(48)

'keychain'

In [105]:
Counter(all_tp_vis_obj_tokens).most_common()

[(30.0, 429),
 (11.0, 283),
 (37.0, 267),
 (7.0, 212),
 (27.0, 199),
 (13.0, 179),
 (18.0, 165),
 (21.0, 159),
 (3.0, 147),
 (17.0, 132),
 (4.0, 122),
 (32.0, 121),
 (1.0, 118),
 (31.0, 117),
 (14.0, 114),
 (8.0, 114),
 (2.0, 110),
 (33.0, 108),
 (15.0, 101),
 (89.0, 90),
 (12.0, 86),
 (10.0, 83),
 (29.0, 82),
 (69.0, 81),
 (63.0, 81),
 (24.0, 77),
 (60.0, 76),
 (25.0, 73),
 (102.0, 65),
 (90.0, 64),
 (26.0, 61),
 (77.0, 60),
 (68.0, 58),
 (20.0, 52),
 (71.0, 51),
 (45.0, 50),
 (76.0, 48),
 (39.0, 46),
 (83.0, 45),
 (91.0, 39),
 (49.0, 37),
 (22.0, 37),
 (34.0, 37),
 (52.0, 36),
 (5.0, 35),
 (75.0, 32),
 (97.0, 32),
 (93.0, 31),
 (94.0, 31),
 (55.0, 31),
 (48.0, 30),
 (6.0, 28),
 (98.0, 27),
 (59.0, 26),
 (43.0, 24),
 (28.0, 23),
 (70.0, 23),
 (65.0, 22),
 (62.0, 21),
 (85.0, 20),
 (78.0, 19),
 (66.0, 19),
 (47.0, 19),
 (74.0, 17),
 (82.0, 14),
 (58.0, 14),
 (64.0, 12),
 (23.0, 12),
 (42.0, 12),
 (61.0, 10),
 (46.0, 10),
 (36.0, 8),
 (54.0, 8),
 (92.0, 5),
 (44.0, 4),
 (105.0, 4),
 (10

In [89]:
np.nonzero(tp_vis)

(array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7]),
 array([ 7, 15, 16, 27, 42,  0,  1, 12, 17, 18, 19, 20, 22, 29, 33, 35, 38,
        45, 46,  0,  1, 12, 15, 17, 18, 19, 20, 21, 22, 29, 33, 35, 38, 45,
        46,  7, 16, 27,  0,  1, 12, 15, 17, 19, 20, 29, 33, 35, 38, 45, 46,
         0,  1, 12, 15, 16, 17, 18, 19, 20, 22, 29, 33, 35, 38, 45, 46]))

In [93]:
tp_vis

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [52]:
mod22_train_sanity[task_id]['p_obj_vis']

{'0': [2.5499094391534527e-08,
  0.004078395664691925,
  0.07042602449655533,
  3.435901518855644e-08,
  3.254684699527388e-08,
  0.017203407362103462,
  0.08790273219347,
  0.009599609300494194,
  0.11370303481817245,
  0.09679776430130005,
  0.12150963395833969,
  0.09460017830133438,
  0.09563472121953964,
  0.12072663009166718,
  0.09913529455661774,
  0.08845728635787964,
  0.07379430532455444,
  0.06536819040775299,
  0.05480298027396202,
  0.04611154645681381,
  0.04492828622460365,
  0.03829255700111389,
  0.05986905097961426,
  0.05650610104203224,
  0.03635989502072334,
  0.02383292093873024,
  0.13524790108203888,
  0.4564535319805145,
  0.299143522977829,
  0.17108577489852905,
  0.07298608869314194,
  0.3198617398738861,
  0.23330341279506683,
  0.31969761848449707,
  0.3277456760406494,
  0.20032791793346405,
  0.16060326993465424,
  1.8977059923486195e-08,
  2.0628339925110595e-08,
  0.04172750562429428,
  0.0649208053946495,
  0.17081530392169952,
  0.09420652687549591,

In [13]:
task_id = 'trial_T20190907_200104_945677_0'
mod22_train_sanity[task_id].keys()

dict_keys(['root', 'action_low', 'action_high', 'p_lang_instr', 'obj_token_id', 'p_obj_vis', 'p_state_change', 'l_obj_vis', 'l_state_change'])

In [16]:
mod22_train_sanity[task_id]['p_state_change'].keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'])

In [28]:
mod22_train_sanity[task_id]['action_high']

['GotoLocation', 'PickupObject', 'GotoLocation', 'PutObject', 'NoOp']

In [26]:
mod22_train_sanity['trial_T20190908_143948_082471_0']['p_state_change']['6']

[]

In [27]:
mod22_train_sanity[task_id]['l_state_change']['3']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0]